\# Developer: Ali Hashaam (ali.hashaam@initos.com) <br>
\# 26th February 2019 <br>

\# © 2019 initOS GmbH <br>
\# License MIT <br>

The code is responsible to get data for ERPNext (https://github.com/frappe/erpnext) from github

In [ ]:
from __future__ import unicode_literals
import urllib2, json, re
import pandas as pd
from bs4 import BeautifulSoup
import logging
import logging.handlers
from github import Github
import pickle

In [6]:
def establish_logger(plateform):
    logger = logging.getLogger(plateform+"git_scrapper")
    #logger.setLevel(logging.INFO)
    fh = logging.FileHandler('scrapping_logs/'+plateform+"git.log")
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    fh.setFormatter(formatter)
    logger.addHandler(fh)
    return logger

In [7]:
def get_data(repo, issues_to_be_done, issues_list):
    try:
        for i in issues_to_be_done:
            issue = repo.get_issue(number=i)
            labels = [l.name for l in issue.get_labels()]
            if 'bug' in labels:
                issue_type = 'bug'
            else:
                issue_type = ','.join(labels)
            comments = issue.get_comments()
            issue_num = issue.number
            issue_body = issue.body
            issue_user = issue.user.login
            issue_state = issue.state
            #comments_list = []
            if comments.totalCount:
                for comment in comments:
                    #comments_list.append(comment)
                    issues_list.append({
                        'id': issue_num,
                        'title': issue_body,
                        'user': issue_user,
                        'type': issue_type,
                        'comment_user': comment.user.login,
                        'comments': comment.body,
                        'status': issue_state
                    })
            else:
                issues_list.append({
                        'id': issue_num,
                        'title': issue_body,
                        'user': issue_user,
                        'type': issue_type,
                        'comment_user': '',
                        'comments': '',
                        'status': issue_state
                    })
            print 'issue {} done'.format(issue_num)
    except Exception as e: 
        print(e)
    return issues_list

In [3]:
data = pd.read_csv('erpnext_issues_seperated_comments.csv')
done_issues = data['id'].tolist()
pickle_file=open('erpnext_issue_list.pkl', 'rb')
issue_list = pickle.load(pickle_file)
issues_to_be_done = set(issue_list)-set(done_issues)

g = Github("*******", "******")
logger = establish_logger('erpnext')
repo = g.get_repo("frappe/erpnext")

issues_list = []

issues_data = get_data(repo, issues_to_be_done, issues_list)

df_seperated_comments =  pd.DataFrame(data = issues_data)
df_seperated_comments = df_seperated_comments.dropna(subset=['title'])
df_final = pd.concat([data, df_seperated_comments])

In [13]:
len(issues_data), len(df_seperated_comments), len(data), len(df_final)

(2550, 2441, 8286, 10727)

In [14]:
df_final.to_csv('erpnext_issues_seperated_comments.csv', index=False,
                             encoding='utf-8')

df_final['type'] = df_final['type'].fillna('')
df_final['comments'] = df_final['comments'].fillna('')
df_final['comments'] = df_final['comments'].str.decode("utf-8")
df_final['title'] = df_final['title'].str.decode("utf-8")
df_final_groupby = df_final.groupby(['id'])['comments'] \
                                .apply(','.join).to_frame('textual_data').reset_index()
df = pd.merge(df_final_groupby, df_final, how='left', on='id')
df['textual_data'] = df['title'].fillna('') + ',' + df['textual_data'].fillna('')
df = df.dropna(subset=['textual_data'])
df[['id', 'type', 'textual_data']].to_csv('erpnext_issues.csv', index=False,
                             encoding='utf-8')